In [39]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")

In [44]:
elo_data = pd.read_sql_query(
    "Select DISTINCT * From AllMatches where Date > '2022-01-01'",
    con=devengine,
)


In [41]:
elo_data=elo_data[elo_data['Sex']=='Mens']
elo_data['Fav']=elo_data.apply(lambda x: x['Player_1'] if  x['Player_1_Odds']<x['Player_2_Odds'] else x['Player_2'],axis=1)
elo_data['Dog']=elo_data.apply(lambda x: x['Player_2'] if  x['Player_1_Odds']<x['Player_2_Odds'] else x['Player_1'],axis=1)
elo_data['Fav_Odds']=elo_data.apply(lambda x: x['Player_1_Odds'] if  x['Player_1_Odds']<x['Player_2_Odds'] else x['Player_2_Odds'],axis=1)
elo_data['Dog_Odds']=elo_data.apply(lambda x: x['Player_2_Odds'] if  x['Player_1_Odds']<x['Player_2_Odds'] else x['Player_1_Odds'],axis=1)
elo_data[['Fav_Odds','Dog_Odds']]=elo_data[['Fav_Odds','Dog_Odds']].astype(float)
elo_data.to_excel('delete.xlsx',index=False)
#elo_data[elo_data['Fav_Odds']==elo_data['Dog_Odds']]

In [42]:
for x in range(2,4):
    for y in range(0,10):
        lower=x+(y*0.1)
        upper=x+((y+1)*0.1)
        #print(lower,upper)
        test1=elo_data[(elo_data['Dog_Odds'].ge(lower))&(elo_data['Dog_Odds'].le(upper))].copy()
        percent=len(test1[test1['Player_1']!=test1['Fav']])/len(test1)
        STAKE=500
        test1['Profit']=test1.apply(lambda x: -STAKE if x['Player_1']==x['Fav'] else (STAKE * x['Dog_Odds'])-STAKE,axis=1)
        if test1['Profit'].sum() >0 and len(test1) >20:
            print(lower,upper,percent)
            print(test1['Profit'].sum())


2.5 2.6 0.4142857142857143
18260.0
3.2 3.3 0.3096774193548387
710.0
3.6 3.7 0.29357798165137616
7970.0
3.7 3.8 0.2755102040816326
3075.0


In [43]:
for x in range(1,4):
    for y in range(0,10):
        lower=x+(y*0.1)
        upper=x+((y+1)*0.1)

        test1=elo_data[(elo_data['Fav_Odds'].ge(lower))&(elo_data['Fav_Odds'].le(upper))].copy()
        if len(test1) > 0:
            percent=len(test1[test1['Player_1']==test1['Fav']])/len(test1)
            STAKE=500
            test1['Profit']=test1.apply(lambda x: -STAKE if x['Player_1']!=x['Fav'] else (STAKE * x['Fav_Odds'])-STAKE,axis=1)
            if test1['Profit'].sum() >0 and len(test1) >20:
                print(lower,upper,percent)
                print(test1['Profit'].sum())
